In [ ]:
!pip install torch_geometric
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
import torch.nn as nn
import random
import time
import matplotlib.pyplot as plt

選擇要使用的資料庫

In [ ]:
# 定義數據集名稱
#dataset = 'CiteSeer'
dataset = 'Cora'
#dataset = 'PubMed'
path = "./"
# 創建 Planetoid 數據集對象，同時進行特徵歸一化的轉換
# Planetoid 是 PyTorch Geometric 提供的常用圖數據集之一
# transform=T.NormalizeFeatures() 表示對節點特徵進行歸一化處理
dataset = Planetoid(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

設定參數、代數、初始解等等

In [ ]:
Ngen=3
Nsol=3
num_experiments=1


In [ ]:
random_number_range=[(1,5), (0,3), (0,3), (150,300), (50, 200), (300, 1000), (8, 256), (8, 256), (8, 256)
                     , (8, 256), (8, 256),(0.2,0.8),(0.001, 0.1),(0.001, 0.1),(1e-5, 1e-2),(1e-5, 1e-2)
                     ,(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2)
                    ,(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2),(1e-5, 1e-2)]
start_sol=[4,2,0,173,176, 1589,106,106,106,106,106,0.7414438607175802, 0.038590004301053006
   , 0.02999691290341772, 0.008409996562194955, 0.008409996562194955
   , 0.008409996562194955, 0.008409996562194955, 0.008409996562194955, 
   0.00408868119073893,0.00408868119073893, 0.00408868119073893, 
   0.00408868119073893,0.00408868119073893, 0.00955616228967521
   , 0.00048096144638867577]

In [ ]:
# 打印節點特徵矩陣的形狀
print(data.x.shape)
print(data.y.shape)
# 打印邊的索引矩陣的形狀
print(data.edge_index.shape)
print(set(data.y.numpy()))

In [ ]:
#創建了一個空的 NetworkX 圖
nxgraph = nx.Graph()
#將資訊添加到 NetworkX 圖中
for x in range(len(data.x)):
    nxgraph.add_node(x)
for u, v in zip(data.edge_index[0].numpy(), data.edge_index[1].numpy()):
    nxgraph.add_edge(u, v)

設可調之網路 optimer都要改

In [ ]:
class Net(nn.Module):
    def __init__(self, num_layers=2, hidden_dims=None, dropout=0.5, activation=1):
        super(Net, self).__init__()
        self.num_layers = num_layers
        self.hidden_dims = hidden_dims if hidden_dims is not None else [16] * num_layers  # 默认隐藏维度为16，如果未指定，则都为16
        self.conv_layers = nn.ModuleList()

        for i in range(num_layers):
            input_dim = dataset.num_features if i == 0 else self.hidden_dims[i-1]  # 使用不同的隐藏维度
            self.conv_layers.append(GCNConv(input_dim, self.hidden_dims[i], cached=True))
        self.fc1 = nn.Linear(self.hidden_dims[-1], dataset.num_classes)
        self.dropout = dropout
        self.activation = activation

    def forward(self):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        for i in range(self.num_layers):
            conv_layer = self.conv_layers[i]
            x = conv_layer(x, edge_index, edge_weight)
            
            if self.activation == 0:
                x = F.tanh(x)
            elif self.activation == 1:
                x = F.relu(x)
            elif self.activation == 2:
                x = F.leaky_relu(x)
            else:
                x = F.tanh(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)


In [ ]:

#設備選擇
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def train1():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs
# 定義訓練函數，包括損失計算和反向傳播
def train2():
    model.train()
    optimizer.zero_grad()
    model_out = model()[data.train_mask]
    y_true = data.y[data.train_mask]
    nllloss = F.nll_loss(model_out, y_true)
    disloss_neg = F.logsigmoid(-1 * (model_out[y_neg_pairs.T[0]]*model_out[y_neg_pairs.T[1]])).sum(-1).mean()
    disloss_pos = F.logsigmoid((model_out[y_pos_pairs.T[0]]*model_out[y_pos_pairs.T[1]])).sum(-1).mean()
    loss = 10 * nllloss - disloss_neg - disloss_pos
    loss.backward()
    optimizer.step()
# 定義測試函數，評估模型性能
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs
# 定義要優化的目標函數。
def objective_function(params):
    data = dataset[0]
    lr, weight_decay ,dropout,hidden_dim,N= params  # 從參數中提取學習率和權重衰減。
    model, data = Net(num_layers=N,dropout=dropout,hidden_dim=hidden_dim).to(device), data.to(device)
    optimizer = torch.optim.Adam([
    {'params': model.conv_layers.parameters(), 'weight_decay': weight_decay},
    {'params': model.fc1.parameters(), 'weight_decay': weight_decay}
    ], lr=lr)
      #訓練迴圈
    best_val_acc = test_acc = 0
    train_losses, val_losses = [], []  # 用於存儲訓練和驗證的LOSS值
    for epoch in range(1, 101):
      train1()
      train_acc, val_acc, tmp_test_acc = test()
      train_loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).item()
      val_loss = F.nll_loss(model()[data.val_mask], data.y[data.val_mask]).item()
      train_losses.append(train_loss)
      val_losses.append(val_loss)

      if val_acc > best_val_acc:
          best_val_acc = val_acc
          test_acc = tmp_test_acc
          epoch_best = epoch
    with torch.no_grad():
      model.eval()
      out = model()
    y_neg_pairs = []
    y_pos_pairs = []
    # 獲取數據索引
    data_idx = np.arange(len(data.x))
    # 遍歷數據集中的標籤
    for idx1, y1 in enumerate(data.y[data.train_mask].cpu().numpy()):
        for idx2, y2 in enumerate(data.y[data.train_mask].cpu().numpy()):
            # 創建負樣本對：當兩個樣本的索引不相等且標籤不相同時，將它們添加到負樣本對列表中
            if idx1 > idx2 and y1 != y2:
                y_neg_pairs.append([idx1, idx2])
            # 創建正樣本對：當兩個樣本的索引不相等且標籤相同時，將它們添加到正樣本對列表中
            if idx1 > idx2 and y1 == y2:
                y_pos_pairs.append([idx1, idx2])
    # 轉換負樣本對和正樣本對列表為NumPy數組
    y_neg_pairs = np.array(y_neg_pairs)
    y_pos_pairs = np.array(y_pos_pairs)
    # 創建並訓練模型，使用給定的學習率和權重衰減。
    model = Net(dropout = dropout).to(device)
    optimizer = torch.optim.Adam([
    {'params': model.conv_layers.parameters(), 'weight_decay': weight_decay},
    {'params': model.fc1.parameters(), 'weight_decay': weight_decay}
    ], lr=lr)

    best_val_acc = 0  # 用於追蹤最佳驗證準確度的變數
    best_model_state_dict = None  # 用於保存最佳模型權重的變數

    for epoch in range(1, 301):
        train2()
        # 計算驗證準確度
        val_acc = test()[1]
        # 更新最佳驗證準確度和最佳模型權重
        if val_acc > best_val_acc:
            best_val_acc = val_acc
    return best_val_acc

#隨機範圍取值
def random_select(n):
    return random.uniform(random_number_range[n][0],random_number_range[n][1])


In [ ]:
model, data = Net().to(device), data.to(device)

In [ ]:
# 调用函数生成随机数
def generate_random_numbers(number_range):
    random_numbers = []
    for min_value, max_value in number_range:
        random_value = random.uniform(min_value, max_value)
        if random_value>1:
          random_value=int(random_value)
        random_numbers.append(random_value)

    return random_numbers
# 定义范围


rX = generate_random_numbers(random_number_range)


In [ ]:
C1=0.4#GBEST區間
C2=0.65#PBEST區間
Cg=0.5
Cp=0.8
Cw=0.9

In [ ]:
Njob=11 # 表離散型參數個數
Njob2=15 # 表連續型參數個數

In [ ]:
#iSSO-ssoHPO（超參數優化）老師版本

import numpy as np
import time
import copy
import random
import os

random.seed()

#random_number_range=[(0.0005, 0.1), (1e-4, 1e-3),(0.2,0.7),(8,256),(1,4)]
# 定義參數。

#代數與每代幾組解
Best_time=0
data = dataset[0]
gbest_list=[]

start_time = time.time()
'''----- 生成初始种群 -----'''
X, FX, pX, pF, gBest, genBest= [], [], [], [], 0, 0,
value_to_x_dict = {}
umax=np.array([0]*Njob, dtype=np.float64)
umin=np.array([1000]*Njob, dtype=np.float64)
#pX存gbest解pF存值
gen=0
#初始解
for sol in range(Nsol):
    rX = generate_random_numbers(random_number_range)
    if sol==Nsol-1:
        rX=list(start_sol)
    X.append(rX) # add to the X2.
    pX.append(rX) # add to the X2.
    for job in range(Njob):
        if X[sol][job]>umax[job]:
            umax[job]=X[sol][job]
        if X[sol][job]<umin[job]:
            umin[job]=X[sol][job]
    best_val_acc_list = []
    
    best_num_layers = int(X[sol][0])
    best_optim = int(X[sol][1])
    best_activation = int(X[sol][2])
    earlystop = int(X[sol][3])
    epoch1 = int(X[sol][4])
    epoch2 = int(X[sol][5])
    best_hidden_dim_list = [X[sol][6+i] for i in range(best_num_layers)]
    best_dropout = X[sol][11]
    best_lr1 = X[sol][12]
    best_lr2 = X[sol][13]
    best_weight_decay_list1 = [X[sol][14+i] for i in range(best_num_layers)]
    best_weight_decay_list2 = [X[sol][19+i] for i in range(best_num_layers)]
    best_weight_decay_fc1 = X[sol][24]
    best_weight_decay_fc2 = X[sol][25]
     
    model, data = Net(hidden_dims=best_hidden_dim_list,dropout = best_dropout,num_layers=best_num_layers,activation=best_activation).to(device), data.to(device)
    optimizer_params1 = []
    for i, conv_layer in enumerate(model.conv_layers):
        optimizer_params1.append({'params': conv_layer.parameters(), 'weight_decay': best_weight_decay_list1[i]})
    optimizer_params1.append({'params': model.fc1.parameters(), 'weight_decay': best_weight_decay_fc1})
    if best_optim==0:
        optimizer = torch.optim.Adam(optimizer_params1, lr=best_lr1)
    elif best_optim==1:
        optimizer = torch.optim.AdamW(optimizer_params1, lr=best_lr1)
    elif best_optim==2:
        optimizer = torch.optim.Adagrad(optimizer_params1, lr=best_lr1)
    elif best_optim==3:
        optimizer = torch.optim.AMSGrad(optimizer_params1, lr=best_lr1)
    best_val_acc = test_acc = 0
    train_losses, val_losses = [], []  # 用於存儲訓練和驗證的LOSS值
    for epoch in range(1, epoch1):
        train1()
        train_acc, val_acc, tmp_test_acc = test()
        train_loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).item()
        val_loss = F.nll_loss(model()[data.val_mask], data.y[data.val_mask]).item()
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            epoch_best = epoch
    with torch.no_grad():
        model.eval()
        out = model()
    y_neg_pairs = []
    y_pos_pairs = []
      # 獲取數據索引
    data_idx = np.arange(len(data.x))
      # 遍歷數據集中的標籤
    for idx1, y1 in enumerate(data.y[data.train_mask].cpu().numpy()):
          for idx2, y2 in enumerate(data.y[data.train_mask].cpu().numpy()):
              # 創建負樣本對：當兩個樣本的索引不相等且標籤不相同時，將它們添加到負樣本對列表中
              if idx1 > idx2 and y1 != y2:
                  y_neg_pairs.append([idx1, idx2])
              # 創建正樣本對：當兩個樣本的索引不相等且標籤相同時，將它們添加到正樣本對列表中
              if idx1 > idx2 and y1 == y2:
                  y_pos_pairs.append([idx1, idx2])
      # 轉換負樣本對和正樣本對列表為NumPy數組
    y_neg_pairs = np.array(y_neg_pairs)
    y_pos_pairs = np.array(y_pos_pairs)
      # 創建神經網絡模型並將其移動到指定設備（通常是GPU）
    for k in range(num_experiments):
        model, data = Net(hidden_dims=best_hidden_dim_list,dropout = best_dropout,num_layers=best_num_layers,activation=best_activation).to(device), data.to(device)
        optimizer_params2 = []
        for i, conv_layer in enumerate(model.conv_layers):
            optimizer_params2.append({'params': conv_layer.parameters(), 'weight_decay': best_weight_decay_list2[i]})
        optimizer_params2.append({'params': model.fc1.parameters(), 'weight_decay': best_weight_decay_fc2})
        if best_optim==0:
            optimizer = torch.optim.Adam(optimizer_params2, lr=best_lr2)
        elif best_optim==1:
            optimizer = torch.optim.AdamW(optimizer_params2, lr=best_lr2)
        elif best_optim==2:
            optimizer = torch.optim.Adagrad(optimizer_params2, lr=best_lr2)
        elif best_optim==3:
            optimizer = torch.optim.AMSGrad(optimizer_params2, lr=best_lr2)
        # 初始化最佳驗證準確率和測試準確率
        best_val_acc = test_acc = 0
        patience = earlystop  # 最大停止等待周期
        train_losses, val_losses,test_acc_list,val_acc_list  =[], [], [],[]  # 用於存儲訓練和驗證的LOSS值
        for epoch in range(1, epoch2):
          train2()
          train_acc, val_acc, tmp_test_acc = test()
          train_loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).item()
          val_loss = F.nll_loss(model()[data.val_mask], data.y[data.val_mask]).item()
          train_losses.append(train_loss)
          val_losses.append(val_loss)
          test_acc_list.append(tmp_test_acc)
          val_acc_list.append(val_acc)
          if val_acc > best_val_acc:
              best_val_acc = val_acc
              test_acc = tmp_test_acc
              epoch_best = epoch  
              wait = 0
          else:
              wait += 1
          if wait >= patience:
            #print(f'Early stopping after {epoch} epochs.',best_val_acc)
            break

        #print(k,epoch,best_val_acc)
        best_val_acc_list.append(best_val_acc)
    value=np.mean(best_val_acc_list)
    FX.append(value)
    pF.append(value)
    if FX[sol]>FX[gBest]:
        gBest=sol
        pX[gBest]=rX
        gbest_list=best_val_acc_list
    print(gen,sol ,rX,value)
    value_to_x_dict[value] = (tuple(X[sol]),(gen,sol))
for gen in range(1,Ngen+1):
  for sol in range(Nsol):
      job=-1
      u = (umax-umin)/2*Njob 
      #isso
      while Njob<=job<Njob+Njob2-1:
        job+=1
        rnd=np.random.rand()
        #print("isso",job,rnd)
        if sol<1:
          #重要!未有初始u只好隨機兩個解之後預計刪除
          X[sol][job]=random_select(job)
        else:
          if rnd<C1 or (X[sol-1][job]==value_to_x_dict[pF[gBest]][0][job]):
            X[sol][job]=X[sol-1][job]+u[job]*generate_random_numbers(random_number_range)[job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                #print("random",random_number_range[job])
                X[sol][job]=random_select(job)
          elif rnd<C2:
            X[sol][job]=pX[gBest][job]+u[job]*generate_random_numbers(random_number_range)[job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                #print("random",random_number_range[job])
                X[sol][job]=random_select(job)
          else:
            X[sol][job]=X[sol-1][job]+(X[sol-1][job]-pX[gBest][job])*generate_random_numbers(random_number_range)[job]
            if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
                #print("random",random_number_range[job])
                X[sol][job]=random_select(job)
        if X[sol][job]>umax[job]:
            umax[job]=X[sol][job]
        if X[sol][job]<umin[job]:
            umin[job]=X[sol][job]
      best_num_layers = int(X[sol][0])
      best_optim = int(X[sol][1])
      best_activation = int(X[sol][2])
      earlystop = int(X[sol][3])
      epoch1 = int(X[sol][4])
      epoch2 = int(X[sol][5])
      best_hidden_dim_list = [X[sol][6+i] for i in range(best_num_layers)]
      best_dropout = X[sol][11]
      best_lr1 = X[sol][12]
      best_lr2 = X[sol][13]
      best_weight_decay_list1 = [X[sol][14+i] for i in range(best_num_layers)]
      best_weight_decay_list2 = [X[sol][19+i] for i in range(best_num_layers)]
      best_weight_decay_fc1 = X[sol][24]
      best_weight_decay_fc2 = X[sol][25]      
      #sso
      X[sol][1]=int(X[sol][1])
      X[sol][2]=int(X[sol][2])
      while job<=Njob-1:
        job+=1

        rnd2=np.random.rand()
        #print("sso",job,rnd2)
        if rnd2<Cg:
          X[sol][job]=pX[gBest][job]
          if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
              #print("random",random_number_range[job])
              X[sol][job]=int(random_select(job))
        elif rnd2<Cp:
          X[sol][job]=pX[sol][job]
          if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
              #print("random",random_number_range[job])
              X[sol][job]=int(random_select(job))
        elif rnd2<Cw:
          X[sol][job]=X[sol-1][job]
          if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
              #print("random",random_number_range[job])
              X[sol][job]=int(random_select(job))
        else:
          X[sol][job]=int(random_select(job))
          if X[sol][job]<random_number_range[job][0] or X[sol][job]>random_number_range[job][1]:
              #print("random",random_number_range[job])
              X[sol][job]=int(random_select(job))
      #算u
      
      #計算value
      best_val_acc_list = []
      break_c=0
      start_time1 = time.time()        
      model, data = Net(hidden_dims=best_hidden_dim_list,dropout = best_dropout,num_layers=best_num_layers,activation=best_activation).to(device), data.to(device)
      optimizer_params1 = []
      for i, conv_layer in enumerate(model.conv_layers):
        optimizer_params1.append({'params': conv_layer.parameters(), 'weight_decay': best_weight_decay_list1[i]})
      optimizer_params1.append({'params': model.fc1.parameters(), 'weight_decay': best_weight_decay_fc1})
      if best_optim==0:
        optimizer = torch.optim.Adam(optimizer_params1, lr=best_lr1)
      elif best_optim==1:
        optimizer = torch.optim.AdamW(optimizer_params1, lr=best_lr1)
      elif best_optim==2:
        optimizer = torch.optim.Adagrad(optimizer_params1, lr=best_lr1)
      elif best_optim==3:
        optimizer = torch.optim.AMSGrad(optimizer_params1, lr=best_lr1)
      best_val_acc = test_acc = 0
      train_losses, val_losses = [], []  # 用於存儲訓練和驗證的LOSS值
      for epoch in range(1, epoch1):
        train1()
        train_acc, val_acc, tmp_test_acc = test()
        train_loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).item()
        val_loss = F.nll_loss(model()[data.val_mask], data.y[data.val_mask]).item()
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            test_acc = tmp_test_acc
            epoch_best = epoch
      with torch.no_grad():
        model.eval()
        out = model()
      y_neg_pairs = []
      y_pos_pairs = []
      # 獲取數據索引
      data_idx = np.arange(len(data.x))
      # 遍歷數據集中的標籤
      for idx1, y1 in enumerate(data.y[data.train_mask].cpu().numpy()):
          for idx2, y2 in enumerate(data.y[data.train_mask].cpu().numpy()):
              # 創建負樣本對：當兩個樣本的索引不相等且標籤不相同時，將它們添加到負樣本對列表中
              if idx1 > idx2 and y1 != y2:
                  y_neg_pairs.append([idx1, idx2])
              # 創建正樣本對：當兩個樣本的索引不相等且標籤相同時，將它們添加到正樣本對列表中
              if idx1 > idx2 and y1 == y2:
                  y_pos_pairs.append([idx1, idx2])
      # 轉換負樣本對和正樣本對列表為NumPy數組
      y_neg_pairs = np.array(y_neg_pairs)
      y_pos_pairs = np.array(y_pos_pairs)
      # 創建神經網絡模型並將其移動到指定設備（通常是GPU）
      for k in range(num_experiments):
        model, data = Net(hidden_dims=best_hidden_dim_list,dropout = best_dropout,num_layers=best_num_layers,activation=best_activation).to(device), data.to(device)
        optimizer_params2 = []
        for i, conv_layer in enumerate(model.conv_layers):
            optimizer_params2.append({'params': conv_layer.parameters(), 'weight_decay': best_weight_decay_list2[i]})
        optimizer_params2.append({'params': model.fc1.parameters(), 'weight_decay': best_weight_decay_fc2})
        if best_optim==0:
            optimizer = torch.optim.Adam(optimizer_params2, lr=best_lr2)
        elif best_optim==1:
            optimizer = torch.optim.AdamW(optimizer_params2, lr=best_lr2)
        elif best_optim==2:
            optimizer = torch.optim.Adagrad(optimizer_params2, lr=best_lr2)
        elif best_optim==3:
            optimizer = torch.optim.AMSGrad(optimizer_params2, lr=best_lr2)
        # 初始化最佳驗證準確率和測試準確率
        best_val_acc = test_acc = 0
        patience = earlystop  # 最大停止等待周期
        train_losses, val_losses,test_acc_list,val_acc_list  =[], [], [],[]  # 用於存儲訓練和驗證的LOSS值
        for epoch in range(1, epoch2):
          train2()
          train_acc, val_acc, tmp_test_acc = test()
          train_loss = F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).item()
          val_loss = F.nll_loss(model()[data.val_mask], data.y[data.val_mask]).item()
          train_losses.append(train_loss)
          val_losses.append(val_loss)
          test_acc_list.append(tmp_test_acc)
          val_acc_list.append(val_acc)
          if val_acc > best_val_acc:
              best_val_acc = val_acc
              test_acc = tmp_test_acc
              epoch_best = epoch  
              wait = 0
          else:
              wait += 1
          if wait >= patience:
            #print(f'Early stopping after {epoch} epochs.',best_val_acc)
            break

        #print(k,epoch,best_val_acc)
        best_val_acc_list.append(best_val_acc)
        if k==5 and np.mean(best_val_acc_list)+0.02<pF[gBest]:
            break
        elif k==10 and np.mean(best_val_acc_list)+0.01<pF[gBest]:
            break
      end_time1 = time.time()
      value=np.mean(best_val_acc_list)
      #計算value結束
      print(gen,"  ",sol, 'new', X[sol],value , 'vs',pF[gBest])
      #print(value,pF[sol],pF[gBest])
      value_to_x_dict[value] = (tuple(X[sol]),(gen,sol))
      #print(value_to_x_dict)
      #判斷大小
      #print("目前gbest",pF[gBest])
      if value>pF[sol]:
        pF[sol]=value
        #pX[sol]=X[sol]
        if value>pF[gBest]:
          gBest=sol
          genBest=gen
          gbest_list=best_val_acc_list
          Best_time=end_time1-start_time1 

end_time = time.time()
sso_time1=end_time-start_time
sol0=value_to_x_dict[pF[gBest]][0]
print("optimal sequence",value_to_x_dict[pF[gBest]][0])
print("optimal value:%f"%pF[gBest])

print("gbest_time:%f"%Best_time)
print("optimal generation number:", value_to_x_dict[pF[gBest]][1][0])
print("optimal sol number:", value_to_x_dict[pF[gBest]][1][1])
print("sso_time:", sso_time1)

In [ ]:
gbest_list

In [ ]:
max_value = np.max(gbest_list)
min_value = np.min(gbest_list)
sorted_data = sorted(gbest_list)

# 獲取第二小的值
second_smallest = sorted_data[1]
std_deviation = np.std(gbest_list)
print("最大值:", max_value)
print("最小值:", second_smallest)
print("平均值:", sum(gbest_list)/num_experiments)
print("標準差:", std_deviation)

第一組